## facematch

Aqui, espero conseguir mesmo que de maneira rudimentar encontrar os vetores que represetam os dois rostos. Para isso, pretendo utilizar o 'pacote' https://github.com/DefTruth/torchlm.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns

import sys
sys.path.append('../')
from facematch import Facematch
%load_ext autotime

time: 205 µs (started: 2024-02-06 11:18:28 -03:00)


In [2]:
df = pd.read_parquet('../base/filtered_scored_data.parquet')

time: 105 ms (started: 2024-02-06 11:18:35 -03:00)


In [38]:
#laod df with 50% of approved True and 50% of approved False
#df2use = df[df['approved'] == True].sample(100)
#df2use = pd.concat([df2use, df[df['approved'] == False].sample(100)])
#df2use = df2use.sample(100)

time: 406 µs (started: 2024-02-06 12:02:52 -03:00)


In [37]:
registration_images = df2use.registration_image.values
comparision_image = df2use.comparision_image.values
similarity = df2use.similarity.values
unico_status = df2use.approved.values

for i in range(0, len(df2use)):
    fig, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(cv2.imread('../base/imagens/' + registration_images[i])[:,:,::-1])
    ax[1].imshow(cv2.imread('../base/imagens/' + comparision_image[i])[:,:,::-1])
    ax[0].set_title(str(i)+ '   '+ str(unico_status[i]) + '   ' + str(similarity[i]))
    plt.savefig('images_2_check/' + str(i) + '.png')
    plt.close()

time: 1min 53s (started: 2024-02-06 12:00:58 -03:00)


In [45]:
real = unico_status[:].copy()
real[17] = not(real[17])
real[18] = not(real[18])
real[26] = not(real[26])
real[31] = not(real[31])
real[49] = not(real[49])
real[66] = not(real[66])
real[91] = not(real[91])
real[93] = not(real[93])
real[99] = not(real[99])

time: 1.12 ms (started: 2024-02-06 13:31:52 -03:00)


In [50]:
similarity_boolean = similarity > 0.6

#print coffunsuion matrix
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(real, similarity_boolean))
print(classification_report(real, similarity_boolean))

print(confusion_matrix(real, unico_status))
print(classification_report(real, similarity_boolean))

[[44  1]
 [15 40]]
              precision    recall  f1-score   support

       False       0.75      0.98      0.85        45
        True       0.98      0.73      0.83        55

    accuracy                           0.84       100
   macro avg       0.86      0.85      0.84       100
weighted avg       0.87      0.84      0.84       100

[[39  6]
 [ 3 52]]
              precision    recall  f1-score   support

       False       0.75      0.98      0.85        45
        True       0.98      0.73      0.83        55

    accuracy                           0.84       100
   macro avg       0.86      0.85      0.84       100
weighted avg       0.87      0.84      0.84       100

time: 65.8 ms (started: 2024-02-06 13:35:53 -03:00)


In [ ]:
#print coffunsuion matrix
